In [10]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='solar')



workbook = xlrd.open_workbook('hotel-list.xls', ignore_workbook_corruption=True)
df = pd.read_excel(workbook)

In [535]:
df = pd.read_csv('CRAWL.csv')

In [536]:
def print_description(index):
    example = df[df.index == index][['descriptions', 'names']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])

In [537]:
print_description(10)

Bạn đủ điều kiện nhận giảm giá Genius tại GREENECO DA LAT HOTEL - Khách sạn Green Eco Đà Lạt! Để tiết kiệm tại chỗ nghỉ này, bạn chỉ cần đăng nhập.
Tọa lạc tại thành phố Đà Lạt, cách Thiền viện Trúc Lâm 2,2 km và Hồ Tuyền Lâm 2,3 km, GreenEco Da Lat Hotel cung cấp chỗ nghỉ với sảnh khách chung và WiFi miễn phí trong toàn bộ khuôn viên cũng như chỗ đỗ xe riêng miễn phí cho khách lái xe. Chỗ nghỉ này nằm trong bán kính 3,5 km từ Quảng trường Lâm Viên, 3,6 km từ Công viên Yersin và 3,6 km từ Hồ Xuân Hương. Nơi đây có dịch vụ phòng, lễ tân 24 giờ và tổ chức các tour du lịch cho khách.
Tất cả phòng nghỉ tại GreenEco Da Lat Hotel được trang bị tủ lạnh, minibar, ấm đun nước, vòi sen, máy sấy tóc, bàn làm việc, ga trải giường và khăn tắm. Mỗi phòng đều được bố trí TV màn hình phẳng, máy điều hòa và phòng tắm riêng với vòi xịt/chậu rửa vệ sinh cùng đồ vệ sinh cá nhân miễn phí. Một số phòng có sân hiên.
Chỗ nghỉ phục vụ bữa sáng à la carte hằng ngày.
Name: GREENECO DA LAT HOTEL - Khách sạn Gree

In [538]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['descriptions'], 20)
df1 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df1.groupby('descriptions').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 words in hotel description before removing stop words')

In [539]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['descriptions'], 20)
df3 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df3.groupby('descriptions').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in hotel description before removing stop words')

In [540]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['descriptions'], 20)
df5 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df5.groupby('descriptions').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description before removing stop words')

In [541]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(4, 4)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['descriptions'], 20)
df6 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df6.groupby('descriptions').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description before removing stop words')

In [550]:
df['descriptions']=df['descriptions'].apply(lambda x: re.sub('.+đăng nhập.','',x))

In [551]:
df['descriptions']=df['descriptions'].str.replace('\n','')
df['descriptions']=df['descriptions'].str.replace('Toạ lạc','')

In [552]:
location=['Đà Lạt', 'Hà Nội', 'TP. Hồ Chí Minh', 'Vũng Tàu', 'Đà Nẵng', 'Phú Quốc', 'Hội An', 'Nha Trang', 'Sa Pa', 'Huế']
DEST = ['-3712045', '-3714993', '-3730078', '-3733750', '-3712125', '-3726177', '-3715584', '-3723998', '-3728113', '-3715887']

In [553]:
df2= pd.DataFrame({'location':location,'DEST':DEST})

In [554]:
df2['DEST'] = df2['DEST'].astype('object')
df2

,location,DEST
0,Đà Lạt,-3712045
1,Hà Nội,-3714993
2,TP. Hồ Chí Minh,-3730078
3,Vũng Tàu,-3733750
4,Đà Nẵng,-3712125
5,Phú Quốc,-3726177
6,Hội An,-3715584
7,Nha Trang,-3723998
8,Sa Pa,-3728113
9,Huế,-3715887


In [555]:
df['DEST'] = df['DEST'].astype('object')

In [557]:
pd.merge(df,df2,how='left',on = ['DEST'])

,links,names,reviews,scores,addresses,descriptions,DEST,location
0,https://www.booking.com/hotel/vn/bazan-hotel-d...,Bazan Hotel Dalat,NaN,"9,1","36 To Hien Thanh , Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...",-3712045,NaN
1,https://www.booking.com/hotel/vn/dalat-wind.vi...,Dalat Wind Hotel,NaN,"8,4","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...","Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...",-3712045,NaN
2,https://www.booking.com/hotel/vn/ttr-central-h...,TTR Central Apart Hotel,NaN,"9,0","4b Hải Thượng, Đà Lạt, Việt Nam","Nằm cách Quảng trường Lâm Viên 1,8 km, TTR Cen...",-3712045,NaN
3,https://www.booking.com/hotel/vn/raon-valley-v...,Raon Villa,NaN,"9,3","44D Trần Khánh Dư, Phường 8, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...",-3712045,NaN
4,https://www.booking.com/hotel/vn/dalat-feliz-h...,Feliz Dalat Homestay,NaN,"9,5",54/13 Phan Đình Phùng (08 Nguyễn Biểu) (Dốc Nh...,Feliz Dalat Homestay cung cấp chỗ nghỉ ở vị tr...,-3712045,NaN
...,...,...,...,...,...,...,...,...
495,https://www.booking.com/hotel/vn/sankofa-villa...,Sankofa Village Hill Resort & Spa,NaN,"8,6","8 Ngoc Ho, Huong Ho, Huong Tra, Huế, Việt Nam",Sankofa Village Hill Resort & Spa tọa lạc tại ...,-3715887,NaN
496,https://www.booking.com/hotel/vn/la-residence-...,"Azerai La Residence, Hue",NaN,"9,2","5 Le Loi Street, Huế, Việt Nam","Azerai La Residence, Hue là biệt thự kiểu thuộ...",-3715887,NaN
497,https://www.booking.com/hotel/vn/mercure-hue-g...,Muong Thanh Holiday Hue Hotel,NaN,"7,9","38 Le Loi, Huế, Việt Nam","Nằm đối diện Bến phà Huế phía bên kia đường, M...",-3715887,NaN
498,https://www.booking.com/hotel/vn/canary-boutiq...,Canary Boutique Hotel,NaN,"8,7","Number 8 Lane 43 Nguyen Cong Tru Street, Huế, ...","Tiếp đón khách với đồ uống chào mừng, Canary B...",-3715887,NaN


In [560]:
df.sample(5)

,links,names,reviews,scores,addresses,descriptions,DEST
27,https://www.booking.com/hotel/vn/himalaya-phoe...,Himalaya Phoenix Dalat Hotel,NaN,"7,6","23 Le Hong Phong, Ward 4, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...",-3712045
310,https://www.booking.com/hotel/vn/la-an-boutiqu...,La An Central Boutique Villa,NaN,"9,0","09 Ngo Quyen, Minh An, Hội An, Việt Nam","Nằm cách Chùa Cầu Nhật Bản 400 m, La An Centra...",-3715584
228,https://www.booking.com/hotel/vn/estrella-bout...,Estrella Boutique Hotel,NaN,"8,7","145 Nguyễn Văn Thoại, Đà Nẵng, Việt Nam","Tọa lạc tại thành phố Đà Nẵng, Estrella Boutiq...",-3712125
428,https://www.booking.com/hotel/vn/chau-long-sap...,Chau Long Sapa 2 Hotel,NaN,"8,0","24 Dong Loi, Sapa, Lao Cai, Sa Pa, Việt Nam","Nằm trong tòa nhà hình tròn đặc biệt, Chau Lon...",-3728113
165,https://www.booking.com/hotel/vn/bobo-boutique...,BOBO Boutique Hotel,NaN,"9,0","Tạ Uyên 09, Vũng Tàu, Việt Nam","Tọa lạc tại thành phố Vũng Tàu, cách Bãi Trước...",-3733750


# XỬ LÝ STOP WORD

In [497]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [498]:
PATH = 'C:/Users/TechCare/Documents/'

In [499]:
stopwords_path = PATH+"vietnamese-stopwords.txt"
stopwords = get_stopwords_list(stopwords_path)

In [500]:
# viết hàm tiền xử lý dữ liệu trên cột overview
def proccessing_data(data):
  #- chuyển về từ thường
  data = data.lower()
  #- Xóa dấu câu, ký tự đặc biệt
  data = re.sub('\W+',' ', data)
  #- xóa stopword
  data =  ' '.join([word for word in data.split() if word not in stopwords])
  #- tách từ
  #data = word_tokenize(data)
  #- nối câu
  #data =  ' '.join([word for word in data])
  return data

In [501]:
from tqdm import tqdm

In [502]:
for i in tqdm(range(0,len(df))):
  try:
    df['descriptions'][i] = proccessing_data(df['descriptions'][i])
  except Exception as ex:
    print(df['descriptions'][i])

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 240.45it/s]


In [559]:
df.sample(5)

,links,names,reviews,scores,addresses,descriptions,DEST
447,https://www.booking.com/hotel/vn/muong-thanh-s...,Muong Thanh Sapa Hotel,NaN,"7,9","44 Ngu Chi Son Street, Sapa, Lao Cai, Sa Pa, V...","Chỉ cách Thị trấn Sapa, Nhà thờ Đá và Chợ đêm ...",-3728113
76,https://www.booking.com/hotel/vn/rex-hanoi.vi....,Rex Hanoi Hotel,NaN,"8,9","42-44 Gia Ngu Street, Hang Bac Ward, Hoan Kiem...","Tọa lạc tại thành phố Hà Nội, cách Nhà hát Múa...",-3714993
199,https://www.booking.com/hotel/vn/choice-vung-t...,Central Hotel by Joi Hospitality,NaN,"8,4","E1-07 Tạ Uyên phường 3, thành phố vũng tàu,Bà ...","Tọa lạc tại thành phố Vũng Tàu, cách Bãi Trước...",-3733750
38,https://www.booking.com/hotel/vn/d-amp-t-villa...,DT Villa,NaN,"8,5","Thửa 811, Hẻm 29, Đường 3 Tháng 4, Phường 3, Đ...","Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...",-3712045
259,https://www.booking.com/hotel/vn/dusit-princes...,Dusit Princess Moonrise Beach Resort,NaN,"9,0","Group 2, Cua Lap, Duong To, Duong Dong, Phú Qu...","Tọa lạc ở đảo Phú Quốc, cách Bãi Dài 300 m, Du...",-3726177


# TẠO MODEL

In [571]:
def recommendations(names, DEST):

    recommended_hotels = []
    links=[]
    addresses =[]
    scores =[]
    descriptions=[]

    
    ks = df[df.names == names]
    
    hotel = df[df.DEST == DEST]
    hotel = pd.concat([ks, hotel], ignore_index = True)
    
    
    #tính tf-idf + cosine_similarities
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4), min_df=0)
    tfidf_matrix = tf.fit_transform(hotel['descriptions'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    #chuyển names thành index
    hotel.set_index('names', inplace = True)
    indices = pd.Series(hotel.index)
    # gettin the index of the hotel that matches the name
    idx = indices[indices == names].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_10_indexes = list(score_series.iloc[0:11].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(hotel.index)[i])
        links.append(list(hotel['links'])[i])
        addresses.append(list(hotel['addresses'])[i])
        scores.append(list(hotel['scores'])[i])
        descriptions.append(list(hotel['descriptions'])[i])
  
    return pd.DataFrame({'links':links,'names':recommended_hotels,'addresses': addresses,'scores':scores,'Description':descriptions})
    #return score_series

In [572]:
df2

,location,DEST
0,Đà Lạt,-3712045
1,Hà Nội,-3714993
2,TP. Hồ Chí Minh,-3730078
3,Vũng Tàu,-3733750
4,Đà Nẵng,-3712125
5,Phú Quốc,-3726177
6,Hội An,-3715584
7,Nha Trang,-3723998
8,Sa Pa,-3728113
9,Huế,-3715887


In [573]:
recommendations('Menora Grand Da Nang',-3723998)

,links,names,addresses,scores,Description
0,https://www.booking.com/hotel/vn/ruby-light-da...,Menora Grand Da Nang,"69-71-73 Do Bi Street, My Khe Beach, Ngu Hanh ...","8,3","Nằm ở thành phố Đà Nẵng, cách Bãi biển Mỹ Khê ..."
1,https://www.booking.com/hotel/vn/emerald-bay-a...,EMERALD BAY HOTEL & SPA,"100/8C Trần Phú, Nha Trang, Việt Nam","8,3","Tọa lạc tại thành phố Nha Trang, MERALD BAY HO..."
2,https://www.booking.com/hotel/vn/aaron.vi.html...,Aaron Hotel,"6/6 Tran Quang Khai Nha Trang, Nha Trang, Việ...","8,7","Quay mặt ra biển ở thành phố Nha Trang, Aaron ..."
3,https://www.booking.com/hotel/vn/merperle-beac...,MerPerle Beach Hotel,"88 A Tran Phu Street, Loc Tho Ward, Nha Trang,...","9,1",MerPerle Beach Hotel là chỗ nghỉ 4 sao quay mặ...
4,https://www.booking.com/hotel/vn/majestic-prem...,Majestic Premium Hotel,"96B5/4 Trần Phú Phường Lộc Thọ, Nha Trang, Viê...","8,6","Tọa lạc tại thành phố Nha Trang, Majestic Prem..."
5,https://www.booking.com/hotel/vn/v-nha-trang-t...,V Hotel Nha Trang,"08 Hung Vuong Street, Loc Tho Ward, Nha Trang,...","8,5","Nằm ở thành phố Nha Trang, cách Bãi biển Nha T..."
6,https://www.booking.com/hotel/vn/la-vague.vi.h...,La Vague Hotel,"08 Trần Phú 08 Trần Phú, Nha Trang, Việt Nam","8,6","Quay mặt ra biển ở thành phố Nha Trang, cách B..."
7,https://www.booking.com/hotel/vn/daphovina.vi....,ASTON Nha Trang City Hotel,"32 Bắc Sơn phường Vĩnh Hải, Tp. Nha Trang, tỉn...","9,3","Tọa lạc tại thành phố Nha Trang, cách Bãi biển..."
8,https://www.booking.com/hotel/vn/ivy-nha-trang...,Ivy Hotel Nha Trang,"127, 127A Hồng Bàng, phường Tân Lập, thành phố...","8,6","Tọa lạc tại thành phố Nha Trang, Ivy Hotel Nha..."
9,https://www.booking.com/hotel/vn/lucky-sun.vi....,Lucky Sun Hotel,"100/8B Trần Phú, Phường Lộc Thọ, Nha Trang, Vi...","8,7","Tọa lạc tại thành phố Nha Trang, cách Bãi biển..."


In [561]:
import pickle

pickle.dump(df,open('df.pkl','wb'))
pickle.dump(recommendations,open('recommendations.pkl','wb'))

In [563]:
import cloudpickle

In [570]:
x = cloudpickle.dumps(recommendations)

TypeError: an integer is required (got type _io.BufferedWriter)

In [569]:
import pickle

f = pickle.loads(x)

print(f('Menora Grand Da Nang',-3723998))

                                               links  \
0  https://www.booking.com/hotel/vn/ruby-light-da...   
1  https://www.booking.com/hotel/vn/emerald-bay-a...   
2  https://www.booking.com/hotel/vn/aaron.vi.html...   
3  https://www.booking.com/hotel/vn/merperle-beac...   
4  https://www.booking.com/hotel/vn/majestic-prem...   
5  https://www.booking.com/hotel/vn/v-nha-trang-t...   
6  https://www.booking.com/hotel/vn/la-vague.vi.h...   
7  https://www.booking.com/hotel/vn/daphovina.vi....   
8  https://www.booking.com/hotel/vn/ivy-nha-trang...   
9  https://www.booking.com/hotel/vn/lucky-sun.vi....   

                        names  \
0        Menora Grand Da Nang   
1     EMERALD BAY HOTEL & SPA   
2                 Aaron Hotel   
3        MerPerle Beach Hotel   
4      Majestic Premium Hotel   
5           V Hotel Nha Trang   
6              La Vague Hotel   
7  ASTON Nha Trang City Hotel   
8         Ivy Hotel Nha Trang   
9             Lucky Sun Hotel   

                   